In [ ]:
%matplotlib inline
from __future__ import print_function

In [ ]:
# If you have not installed the samwich package, e.g., using:
#   python setup.py install
# then we need to add the repo to our path.
import os,sys
sys.path.append(os.path.join(os.environ['HOME'],'waketracking'))

# tracker_comparison
Written by [Eliot Quon](mailto:eliot.quon@nrel.gov) -- 2017-01-04

Instantaneous snapshot provided by [Paula Doubrawa](mailto:paula.doubrawa@nrel.gov)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from samwich.dataloaders import planar_data
from samwich.waketrackers import track

In [ ]:
trackerlist = track()

## Case-specific parameters

In [ ]:
#kind = 'mean'  # sanity check, not very interesting
kind = 'instantaneous'

In [ ]:
D = 27.0  # to define the search range, and the reference area for the contour methods
zhub = 32.1  # hub height [m], for estimating the freestream reference velocity
aref = 0.3  # induction, for estimating the momentum theory mass/momentum flux

In [ ]:
ref_area = np.pi*D**2/4
print(ref_area)

In [ ]:
ref_CT = 4*aref*(1-aref)  # thrust coefficient
ref_CP = 4*aref*(1-aref)**2  # power coefficient
print(ref_CT,ref_CP)

In [ ]:
styles = {
    'gaussian (R)': {
        'label':r'1-D Gaussian ($\sigma=R$)',
        'color':'b',
        'linestyle':'--','linewidth':2,
        'marker':'o','markersize':6
    },
    'gaussian (1.5R)': {
        'label':r'1-D Gaussian ($\sigma=1.5R$)',
        'color':'m',
        'linestyle':'--','linewidth':2,
        'marker':'o','markersize':6
    },
    'gaussian (Porte-Agel)': {
        'label':r'1-D Gaussian (Porte-Agel model)',
        'color':'r',
        'linestyle':'--','linewidth':2,
        'marker':'o','markersize':6
    },
    'gaussian (ideal)': {
        'label':r'1-D Gaussian (x-mom deficit $\sigma$)',
        'color':'m',
        'linestyle':'--','linewidth':2,
        'marker':'o','markersize':6
    },
    'elliptical': {
        'label':r'2-D Gaussian (opt $\sigma_x$,$\sigma_y$,$\theta$)',
        'color':'y',
        'linestyle':'--','linewidth':2,
        'marker':'o','markersize':6
    },
    
    'const area': {
        'label':r'const area contour',
        'color':'b',
        'linestyle':'-','linewidth':2,
        'marker':'+','markersize':10
    },
    'momentum deficit': {
        'label':r'const x-mom deficit contour',
        'color':'r',
        'linestyle':'-','linewidth':2,
        'marker':'+','markersize':10
    },
    
    'min power': {
        'label':r'min power region',
        'color':'0.6',
        'linestyle':'-','linewidth':2,
        'marker':'x','markersize':8
    },
}

## Read in test data
Note: General dataloaders exist to handle processing of multiple sampling locations and/or many samples over time.

In [ ]:
varlist = ['x','y','z','u','v','w']
sample = planar_data({v: np.loadtxt('MWE-data/3D_{}_{}_WFoR.txt'.format(kind,v)) for v in varlist})

## Calculate freestream
Note: This is an optional step if upstream data are available.

In [ ]:
data = {v: np.loadtxt('MWE-data/freestream_mean_{}_WFoR.txt'.format(v)) for v in varlist}
free_z = data['z'][0,:]
free_Uprofile = np.mean(data['u'],axis=0)
jhub = np.argmin(np.abs(data['y'][:,0]-np.mean(data['y'][:,0])))
khub = np.argmin(np.abs(data['z'][0,:]-zhub))
ref_velocity = data['u'][jhub,khub]
#free_const = free_Uprofile*0.0 + ref_velocity

fig,ax = plt.subplots()
ax.plot(free_Uprofile,free_z,'-k')
ax.axvline(ref_velocity,linestyle='--')
plt.xlabel('u (m/s)')
plt.ylabel('z (m)')

print("ref velocity (at z={z}) is {Uref}".format(z=data['z'][0,khub],Uref=ref_velocity))

In [ ]:
ref_thrust = ref_CT * 0.5*ref_velocity**2 * ref_area  # force / density
print('ref thrust (momentum deficit) is',ref_thrust,'N/(kg/m^3)')

## <font color='grey'>*Create all wake tracking objects at once*</font>

In [ ]:
#
# CLEANER/CLEARER TO JUST CREATE THE TRACKER WITH EACH CASE
#
#for tracker in trackerlist:
#    wake[tracker] = track(sample.sliceI(),method=tracker,verbose=True) 
#    wake[tracker].remove_shear(wind_profile=free_Uprofile)

In [ ]:
wake = {}
yc,zc = {},{}

## Constant area contours

In [ ]:
tracker = 'const area'
wake[tracker] = track(sample.sliceI(),method='ConstantArea',verbose=True) 
wake[tracker].remove_shear(wind_profile=free_Uprofile)
yc[tracker],zc[tracker] = wake[tracker].find_centers(ref_area,
                                                     tol=0.01,  # min spacing to test (default:0.01)
                                                     verbosity=0,  # >0 for debugging
                                                    )  # center is weighted by velocity deficit by default

## Momentum deficit contours

In [ ]:
tracker = 'momentum deficit'
#func = lambda u: u*(ref_velocity-u)  # this neglects shear
func = lambda u,u_tot: -u*u_tot  # == -(U(t,y,z) - U_inf(z))*U(t,y,z) == U*(U_inf - U)
wake[tracker] = track(sample.sliceI(),method='ConstantFlux',verbose=True) 
wake[tracker].remove_shear(wind_profile=free_Uprofile)
yc[tracker],zc[tracker] = wake[tracker].find_centers(ref_thrust,
                                                     flux_function=func,
                                                     #field_names=('u_tot'),  # instantaneous velocity
                                                     field_names=('u','u_tot'),
                                                     tol=0.01,  # min spacing to test (default:0.01)
                                                     verbosity=0,  # >0 for debugging
                                                    )  # center is weighted by velocity deficit by default

## Gaussian Fits

In [ ]:
tracker = 'gaussian (R)'
wake[tracker] = track(sample.sliceI(),method='Gaussian',verbose=True) 
wake[tracker].remove_shear(wind_profile=free_Uprofile)
yc[tracker],zc[tracker] = wake[tracker].find_centers(umin=None,  # default: detected from wake snapshot
                                                     sigma=D/2,
                                                    )  # center is weighted by velocity deficit by default

In [ ]:
tracker = 'gaussian (Porte-Agel)'
from samwich.gaussian_functions import PorteAgel
#xref = np.mean(sample.x) - x0  # need to know turbine location...
xloc = 3*D  # samples 3D downstream

# model depends on case-dependent wake growth rate, kstar
fernando = PorteAgel(CT=ref_CT,d0=D,kstar=0.03) # ad-hoc value taken from Bastankhah & Porte-Agel 2014 (the smallest of the full-scale cases)
#fernando = PorteAgel(CT=ref_CT,d0=D,TI=0.20)

wake[tracker] = track(sample.sliceI(),method='Gaussian',verbose=True) 
wake[tracker].remove_shear(wind_profile=free_Uprofile)
yc[tracker],zc[tracker] = wake[tracker].find_centers(umin=fernando.amplitude(xloc,-ref_velocity),
                                                     sigma=fernando.sigma(xloc),
                                                    )  # center is weighted by velocity deficit by default

In [ ]:
tracker = 'gaussian (ideal)' # sigma estimated from momentum deficit
wake[tracker] = track(sample.sliceI(),method='Gaussian',verbose=True) 
wake[tracker].remove_shear(wind_profile=free_Uprofile)

max_VD = -np.min(wake[tracker].u,axis=(1,2))  # max velocity deficit, u.shape == (Ntimes,Nh,Nv)
sigma_opt = np.sqrt(ref_thrust / (np.pi*max_VD*(2*ref_velocity - max_VD)))

yc[tracker],zc[tracker] = wake[tracker].find_centers(umin=None,  # default: detected from wake snapshot
                                                     sigma=sigma_opt,
                                                    )  # center is weighted by velocity deficit by default

### 2D Gaussian

In [ ]:
tracker = 'elliptical'
wake[tracker] = track(sample.sliceI(),method='Gaussian2D',verbose=True) 
wake[tracker].remove_shear(wind_profile=free_Uprofile)
yc[tracker],zc[tracker] = wake[tracker].find_centers(umin=None,  # default: detected from wake snapshot
                                                     A_min=ref_area/5.0,  # ad hoc value
                                                     A_max=ref_area*2.0,  # ad hoc value
                                                     AR_max=10.0,  # ad hoc value
                                                     verbosity=1,  # > 0 for debugging
                                                    )  # center is weighted by velocity deficit by default

## Test Region Approach

In [ ]:
tracker = 'min power'
wake[tracker] = track(sample.sliceI(),method='CircularTestRegion',verbose=True) 
wake[tracker].remove_shear(wind_profile=free_Uprofile)
yc[tracker],zc[tracker] = wake[tracker].find_centers(test_radius=D/2,  # following Vollmer 2016
                                                     test_function=lambda u: u**3,
                                                     test_field='u_tot',
                                                     #verbosity=2,  # verbosity > 0 for debug output
                                                    )  # center is weighted by velocity deficit by default

## Comparisons

In [ ]:
itime = 0

In [ ]:
for tracker,wakeobj in wake.items():
    print(tracker,'center at',(yc[tracker][itime],zc[tracker][itime]))

In [ ]:
all_yc = [ yc[tracker][itime] for tracker in wake.keys() ]
all_zc = [ zc[tracker][itime] for tracker in wake.keys() ]
yc_range = np.array([np.min(all_yc), np.max(all_yc)])
zc_range = np.array([np.min(all_zc), np.max(all_zc)])
print('y range:',yc_range,'==',yc_range/D,'D')
print('z range:',zc_range,'==',zc_range/D,'D')

### Compare all methods

In [ ]:
c1,c2,dc = -4, 4, 0.1 # for freestream profile

fig,ax = plt.subplots(figsize=(9,6))
p1 = ax.contourf(wake['const area'].xh[:,:],
                 wake['const area'].xv[:,:],
                 wake['const area'].u[0,:,:],levels=np.arange(c1,c2+dc,dc))
plt.colorbar(p1)
ax.set_xlabel('Y (m)')
ax.set_ylabel('Z (m)')

for tracker,wakeobj in wake.items():
    print('Plotting',tracker,styles[tracker]['label'])    
    ax.plot(yc[tracker][itime], zc[tracker][itime],
            color=styles[tracker]['color'],
            marker=styles[tracker]['marker'],linestyle='',
            ms=styles[tracker]['markersize'],mew=2,
            label=styles[tracker]['label'])
    ax.plot(wakeobj.paths[itime][:,0], wakeobj.paths[itime][:,1],
            color=styles[tracker]['color'],
            linestyle=styles[tracker]['linestyle'],
            linewidth=styles[tracker]['linewidth'])
plt.axis('equal')    
legend = ax.legend(loc='center left',bbox_to_anchor=(1.2,0.5),ncol=1)
plt.savefig('all_{:s}_wakes.png'.format(kind), dpi=150,
            bbox_extra_artists=(legend,), bbox_inches='tight')  # to handle outside legend

### Compare contour methods

In [ ]:
wakeobj = wake['const area']

fig,ax = plt.subplots(figsize=(9,6))
p1 = ax.contourf(wakeobj.xh[:,:],
                 wakeobj.xv[:,:],
                 wakeobj.u[0,:,:],levels=np.arange(c1,c2+dc,dc))
ax.plot(wake['const area'].paths[itime][:,0], wake['const area'].paths[itime][:,1],
        color=styles['const area']['color'],
        linestyle=styles['const area']['linestyle'],
        linewidth=styles['const area']['linewidth'])
plt.colorbar(p1)
ax.set_title('U - Umean')

fig,ax = plt.subplots(figsize=(9,6))
p1 = ax.contourf(wakeobj.xh[:,:],
                 wakeobj.xv[:,:],
                 -wakeobj.u[0,:,:]*wakeobj.u_tot[0,:,:])
ax.plot(wake['momentum deficit'].paths[itime][:,0], wake['momentum deficit'].paths[itime][:,1],
        color=styles['momentum deficit']['color'],
        linestyle=styles['momentum deficit']['linestyle'],
        linewidth=styles['momentum deficit']['linewidth'])
plt.colorbar(p1)
ax.set_title('U*(Umean - U)')

fig,ax = plt.subplots(figsize=(9,6))
p1 = ax.contourf(wakeobj.xh[:,:],
                 wakeobj.xv[:,:],
                 wakeobj.u_tot[0,:,:])
ax.plot(wake['const area'].paths[itime][:,0], wake['const area'].paths[itime][:,1],
        color=styles['const area']['color'],
        linestyle=styles['const area']['linestyle'],
        linewidth=styles['const area']['linewidth'],
        label=styles['const area']['label'])
ax.plot(wake['momentum deficit'].paths[itime][:,0], wake['momentum deficit'].paths[itime][:,1],
        color=styles['momentum deficit']['color'],
        linestyle=styles['momentum deficit']['linestyle'],
        linewidth=styles['momentum deficit']['linewidth'],
        label=styles['momentum deficit']['label'])
plt.colorbar(p1)
plt.axis('equal')
ax.legend(loc='best')
ax.set_xlabel('Y (m)')
ax.set_ylabel('Z (m)')
fig.savefig('compare_contour_{}.png'.format(kind),dpi=150)